In [35]:
import pandas as pd
import numpy as np
import re

In [16]:
lines = sc.textFile('bem_candidato_2016_CE_utf8.txt')

In [17]:
lines.count()

26155

In [21]:
lines.first()

'"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001105";"21";"Veículo automotor terrestre: caminhão, automóvel, moto, etc.";"UMA MOTO HONDA BIZ 2005";"2000";"08/09/2016";"18:43:44"'

In [22]:
os_mota = lines.filter(lambda line: 'MOTA' in line)

In [23]:
os_mota.first()

'"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001601";"11";"Apartamento";"UM APTO SITO A RUA DEUSDEDIT COSTA SOUSA,N°55, FORT CE,FINANCIOADO PELA CAIXA ECONOMICA FEDERAL,JUNTO A CONSTRUTORA MOTA MACHADO";"131124.37";"04/09/2016";"16:20:49"'

In [30]:
os_vieira = lines.filter(lambda line: 'VIEIRA' in line)

In [31]:
os_vieira_ou_mota = os_vieira.union(os_mota)

In [32]:
os_vieira_ou_mota.first()

'"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001518";"13";"Terreno";"IMÓVEL LOCALIZADO NA VILA PADRE VIEIRA, S/N, VIÇOSA DO CEARÁ - CE";"35000";"04/09/2016";"16:20:49"'

In [34]:
for line in os_vieira_ou_mota.take(10):
    
    print(line.split(';')[7])

"Terreno"
"Casa"
"Terreno"
"Casa"
"Terreno"
"Quotas ou quinhões de capital"
"Casa"
"Outros bens móveis"
"Quotas ou quinhões de capital"
"Outras participações societárias"


In [42]:
s = 'abelardo v'

In [50]:
re_valor_com_decimal = re.compile(r'\d*\.\d+')
def valor_com_decimal(line):
    
    return re_valor_com_decimal.match(line.split(';')[-3][1:-1])

In [51]:
os_decimal = lines.filter(valor_com_decimal)

In [52]:
os_decimal.take(10)

['"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001115";"12";"Casa";"CASA NO CONDOMINIO LAGOA SECA-MINHA CASA MINHA VIDA";"11978.38";"04/09/2016";"16:20:49"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001140";"41";"Caderneta de poupança";"POPUPANÇA BB  - FORTALEZA, AGENCIA 3140, CONTA 27699-3";"1454.64";"04/09/2016";"16:20:49"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001140";"49";"Outras aplicações e Investimentos";"BB OUROCAP TORCIDA";"30942.36";"04/09/2016";"16:20:49"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001141";"19";"Outros bens imóveis";"COMPRA DE TERRENO NO VALOR DE 20.000,00 MEDINDO 24,00M X 30,00M NA CIDADE DE CRATEUS/CE";"76277.5";"04/09/2016";"16:20:49"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000001141";"19";"Outros bens imóveis";"COMPRA DE APTO Nº 1401, TIPO B, LOCALIZADO NO EDIFICIO PORTE DE LYON, NA RUA NUNES VALENTE  2500, DI

In [55]:
columns = lines.map(lambda line: [col[1:-1] for col in line.split(';')])

In [56]:
columns.take(2)

[['09/09/2016',
  '19:31:01',
  '2016',
  'Eleições Municipais 2016',
  'CE',
  '60000001105',
  '21',
  'Veículo automotor terrestre: caminhão, automóvel, moto, etc.',
  'UMA MOTO HONDA BIZ 2005',
  '2000',
  '08/09/2016',
  '18:43:44'],
 ['09/09/2016',
  '19:31:01',
  '2016',
  'Eleições Municipais 2016',
  'CE',
  '60000001105',
  '61',
  'Depósito bancário em conta corrente no País',
  'CONTA CORRENTE NO BANCO DO BRASIL',
  '1',
  '08/09/2016',
  '18:43:44']]

In [83]:
columns.sample(False, .001).take(1)

[['09/09/2016',
  '19:31:01',
  '2016',
  'Eleições Municipais 2016',
  'CE',
  '60000001495',
  '51',
  'Crédito decorrente de empréstimo',
  'CRETIDO',
  '150000',
  '08/09/2016',
  '16:17:11']]

## parallelize

In [63]:
a = sc.parallelize(np.arange(3))

In [64]:
a.count()

3

## Pseudo set operations

In [65]:
b = a.flatMap(lambda v: [v, v**2, v**3])

In [66]:
b.collect()

[0, 0, 0, 1, 1, 1, 2, 4, 8]

### distinct

In [71]:
b.distinct().collect()

[0, 8, 4, 1, 2]

### union

In [68]:
c = sc.parallelize(np.arange(6))

In [76]:
c.collect()

[0, 1, 2, 3, 4, 5]

In [72]:
c.union(b).collect()

[0, 1, 2, 3, 4, 5, 0, 0, 0, 1, 1, 1, 2, 4, 8]

### intersection

In [74]:
c.intersection(b).collect()
# also removes all duplicates (including duplicates from a single RDD) while running

[0, 1, 2, 4]

### subtract

In [75]:
c.subtract(b).collect()

[3, 5]

### cartesian

In [78]:
c.cartesian(b).take(4)

[(0, 0), (0, 0), (0, 0), (0, 1)]

## Actions

In [85]:
lista_enorme_pra_caramba = sc.parallelize(np.arange(30000))

In [86]:
lista_enorme_pra_caramba.reduce(lambda x,y: x+y)

449985000

In [87]:
c.reduce(lambda x,y: x+y)

15

In [92]:
c.fold(3, lambda x,y: x+y)

30

In [98]:
lines.aggregate(0, lambda acc, line: acc + len(line), lambda acc1,acc2: acc1+acc2)

5292370

In [99]:
lines.top(2)

['"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000017097";"32";"Quotas ou quinhões de capital";"100% DAS QUOTAS DE CAPITAL SOCIAL DA EMPRESA VALERIANO MOURAO MORORO EPP";"10000";"08/09/2016";"19:17:03"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000017082";"64";"Dinheiro em espécie - moeda estrangeira";"DINHEIRO EM ESPECIE";"500";"06/09/2016";"19:22:52"']

In [100]:
lines.top(2, key=len)

['"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000003852";"21";"Veículo automotor terrestre: caminhão, automóvel, moto, etc.";"UM VEICULO ANO 2015/ FIAT PALIO FIRE 2015 - ANO 2015 - DE COR VERMELHO, VALOR DE 34.031,00, SENDO A ENTRADA DE 8.472,20 DE RECURSOS PRÓPRIOS E O RESTANTE FINANCIADO PELA CAIXA ECONÔMICA FEDERAL EM 36";"19406.86";"08/09/2016";"12:37:18"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000009858";"21";"Veículo automotor terrestre: caminhão, automóvel, moto, etc.";"VEICULO AMAROK CD 4X4 HIGH, ADQUIRIDO NA EMPRESA: SAGANOR NORDESTE COMERCIO AUTOMOVEIS E SERVIÇOS LTDA, CNPJ: 07.235.484/0001-05, POR R$ 105.000,00 ENTRADA DE R$ 40.000 E 48 PARCELAS DE R$ 1.950,75";"108246.25";"04/09/2016";"16:20:49"']

In [101]:
lines.takeSample(False, 2, 1)

['"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000003700";"13";"Terreno";"UM TERRENO SITUADO EM TIANGUÁ-CE NA RUA BENJAMIN CAVALCANTE, BAIRRO DO PLANALTO, COM ÁREA TOTAL DE 136M²";"5000";"06/09/2016";"10:35:40"',
 '"09/09/2016";"19:31:01";"2016";"Eleições Municipais 2016";"CE";"60000011955";"61";"Depósito bancário em conta corrente no País";"SALDO DISPONIVÉL EM 31/07/2016 NO BANCO DO BRASIL C/C: 5.283-3 AG: 4050-9";"10523";"06/09/2016";"14:38:33"']

In [103]:
c.foreach(print) # rs, aparece no console

In [104]:
columns.map(lambda cols: cols[7]).countByValue()

defaultdict(int,
            {'Aeronave': 1,
             'Apartamento': 715,
             'Aplicação de renda fixa (CDB, RDB e outros)': 262,
             'Ações (inclusive as provenientes de linha telefônica)': 59,
             'Bem relacionado com o exercício da atividade autônoma': 87,
             'Benfeitorias': 25,
             'Caderneta de poupança': 753,
             'Casa': 3831,
             'Construção': 37,
             'Consórcio não contemplado': 113,
             'Crédito decorrente de alienação': 8,
             'Crédito decorrente de empréstimo': 38,
             'Depósito bancário em conta corrente no País': 946,
             'Depósito bancário em conta corrente no exterior': 2,
             'Dinheiro em espécie - moeda estrangeira': 45,
             'Dinheiro em espécie - moeda nacional': 2233,
             'Direito de autor, de inventor e patente': 4,
             'Direito de lavra e assemelhado': 1,
             'Embarcação': 41,
             'Fundo de ações, inc

In [107]:
columns.takeOrdered(2, lambda cols: -1*float(cols[-3]))

[['09/09/2016',
  '19:31:01',
  '2016',
  'Eleições Municipais 2016',
  'CE',
  '60000007189',
  '32',
  'Quotas ou quinhões de capital',
  'QUOTAS NA EMPRESA CMACK',
  '20000000',
  '06/09/2016',
  '10:56:30'],
 ['09/09/2016',
  '19:31:01',
  '2016',
  'Eleições Municipais 2016',
  'CE',
  '60000011679',
  '32',
  'Quotas ou quinhões de capital',
  'QUOTAS DE CAPITAL (JUNTO C/ CONJUGUE) COMPESCAL COM. DE PESC. ARAC. LTDA -  CGC 07.108.145/0001-50 CAPITAL SOCIAL AUMENTADO COM LUCROS ACUMULADOS',
  '15993630',
  '06/09/2016',
  '18:22:59']]

## Persistence (Caching)

In [155]:
el_ricoes = columns.map(lambda cols: float(cols[-3]) / 12345)

In [156]:
%%time
el_ricoes.count()

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 169 ms


26155

In [157]:
el_ricoes.persist(storageLevel=StorageLevel(False, True, False, False, 1))

PythonRDD[150] at RDD at PythonRDD.scala:48

In [160]:
%%time
el_ricoes.count()

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 66 ms


26155

In [161]:
el_ricoes.unpersist()

PythonRDD[150] at RDD at PythonRDD.scala:48

In [162]:
%%time
el_ricoes.count()

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 149 ms


26155